In [ ]:
#importing required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
from model import Net # importing the best model created 

In [ ]:
# setting up constants
EPOCHS = 20
BATCH_SIZE = 64

In [ ]:
# using gpu if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

In [ ]:
# setting the transforms and normalizing the images
train_transforms = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)),
                        transforms.RandomRotation((-7.0, 7.0)),
                        transforms.RandomAffine(0, (0.05, 0.05))
])
test_transforms = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
# creating train and test dataset
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [ ]:
SEED = 22

cuda = torch.cuda.is_available()
print(cuda)

torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=BATCH_SIZE)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)


In [ ]:
train_data = train.data
train_data = train.transform(train_data.numpy())

print('[Train]')
print(' - Numpy Shape:', train.data.cpu().numpy().shape)
print(' - Tensor Shape:', train.data.size())
print(' - min:', torch.min(train_data))
print(' - max:', torch.max(train_data))
print(' - mean:', torch.mean(train_data))
print(' - std:', torch.std(train_data))
print(' - var:', torch.var(train_data))

dataiter = iter(train_loader)
images, labels = next(dataiter)

print(images.shape)
print(labels.shape)

# Let's visualize some of the images
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(images[0].numpy().squeeze(), cmap='gray_r')

In [ ]:
figure = plt.figure()
num_of_images = 60
for index in range(1, num_of_images + 1):
    plt.subplot(6, 10, index)
    plt.axis('off')
    plt.imshow(images[index].numpy().squeeze(), cmap='gray_r')

### train and test functions

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch, l1_loss = False, lambda_l1= 0.01):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    # if L1 regularization
    if l1_loss == True:
        l1 = 0
        for p in model.parameters():
            l1 = l1 + p.abs().sum()
        loss = loss + lambda_l1 * l1
    
    train_losses.append(loss)
    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

misclassified = []
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            idxs_mask = ((pred == target) == False).nonzero()
            misclassified.append(data[idxs_mask].numpy())
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

#### Plotting funtions for test loss and test accuracy for the below models

In [ ]:
def plot_loss(ln_loss, gn_loss, bn_loss):
    epochs = [i for i in range(EPOCHS)]
    plt.plot(x=epochs, y=ln_loss, label="Layer Norm")
    plt.plot(x=epochs, y=gn_loss, label="Group Norm")
    plt.plot(x=epochs, y=bn_loss, label="Batch Norm + L1")
    plt.xlabel('EPOCHS')
    plt.ylabel('Loss')
    plt.grid()
    plt.title("Losses for all 3 models")
    plt.legend()
    plt.show()

def plot_acc(ln_acc, gn_acc, bn_acc):
    epochs = [i for i in range(EPOCHS)]
    plt.plot(x=epochs, y=ln_acc, label="Layer Norm")
    plt.plot(x=epochs, y=gn_acc, label="Group Norm")
    plt.plot(x=epochs, y=bn_acc, label="Batch Norm + L1")
    plt.xlabel('EPOCHS')
    plt.ylabel('Accuracy')
    plt.title("Accuracy for all 3 models")
    plt.grid()
    plt.legend()
    plt.show()

## Layer Normalization

In [ ]:
model_ = Net(norm='ln').to(device)
summary(model_, input_size=(1, 28, 28))

In [ ]:
model =  Net(norm='ln').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.3, momentum=0.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

In [ ]:
ln_loss = test_losses
ln_acc = test_acc

## Group Normalization

In [ ]:
model_ = Net(norm='gn').to(device)
summary(model_, input_size=(1, 28, 28))

In [ ]:
model =  Net(norm='gn').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.3, momentum=0.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

In [ ]:
gn_loss = test_losses
gn_acc = test_acc

## Batch Normalization + L1

In [ ]:
model_ = Net(norm='bn').to(device)
summary(model_, input_size=(1, 28, 28))

In [ ]:
model =  Net(norm='bn').to(device)
optimizer = optim.SGD(model.parameters(), lr=0.3, momentum=0.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch, l1_loss=True)
    scheduler.step()
    test(model, device, test_loader)

In [ ]:
bn_loss = test_losses
bn_acc = test_acc